In [1]:
import os
import sys
import torch
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
import networkx as nx
import glob
import pandas as pd
import torch
from torch_geometric.data import InMemoryDataset, download_url

In [190]:
# Load in label by name
def get_label(file, labels):
    pair_1 = file.split('/')[-1]
    pair_1, pair_2 = pair_1.split("and")
    pair_1 = pair_1.replace(".gpickle", "")
    pair_2 = pair_2.replace(".gpickle", "")
    l = int(labels.loc[(labels.protein_1 == pair_1)& (labels.protein_2 == pair_2)].label)
    return file, l

In [202]:

class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        # Download to `self.raw_dir`.
        download_url(url, self.raw_dir)
        ...

    def process(self):
        # Read data into huge `Data` list.
        data_list = [...]

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [191]:
# Import the data
graph_dir_path = '/home/cluster/dsumne/data/geometric/graph_data'
labels_dir_path = '/home/cluster/dsumne/data/geometric/graph_labels'

graph_files = glob.glob(os.path.join(graph_dir_path, '*'))
graph_labels = glob.glob(os.path.join(labels_dir_path, '*'))
graph_labels = pd.read_csv(graph_labels[0])

# Create positive and negative sets
positives = []
pos_labels = []

negatives = []
neg_labels = []

for i, file in enumerate(graph_files):
    obs, label = get_label(file,labels)
    
    if label == 1:
        positives.append(obs)
        pos_labels.append([1,0])
    else:
        negatives.append(obs)
        neg_labels.append([0,1])
